In [16]:
import os
import json
import csv

input_folder = "/Users/kedarjoshi/PyCharmMiscProject/No_Backbone"

# MANE GTF used to validate/select MANE transcripts (transcript_id -> biotype lookup).
mane_gtf_file = "MANE.GRCh38.v1.4.refseq_genomic.gtf"

def load_mane_transcripts(gtf_path):
    """
    We only parse 'transcript' lines and extract 'transcript_id' and 'transcript_biotype'.
    """
    mane_dict = {}
    with open(gtf_path, "r") as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            fields = line.rstrip("\n").split("\t")

            if len(fields) < 9 or fields[2] != "transcript":
                continue
            attrs = {}

            for kv in fields[8].split(";"):
                kv = kv.strip()
                if kv and " " in kv:
                    k, v = kv.split(" ", 1)
                    attrs[k] = v.strip().strip('"')
            tid = attrs.get("transcript_id")
            biotype = attrs.get("transcript_biotype") or "unknown"
            if tid:
                mane_dict[tid] = biotype
    # Print a short summary so user knows how many transcripts were loaded
    print(f"Loaded {len(mane_dict)} MANE transcripts from {gtf_path}")
    return mane_dict

def safe_join_list(l, sep=","):
    """
    Helper to join list elements into a string safely.
    """
    return sep.join([str(x) for x in l]) if l else ""

def select_transcript(transcripts, mane_dict):
    """
    Choose the single transcript to report for a variant.

    Priority:
      1) transcript has isManeSelect True, biotype == "mRNA", and exists in MANE GTF
      2) transcript has biotype == "mRNA" and isCanonical True
      3) fallback: the first transcript in the list

    Returns a single transcript dict or None if transcripts list is empty.
    """
    if not transcripts:
        return None
    # Step 1: prefer MANE Select + mRNA
    for tx in transcripts:
        tx_id = tx.get("transcript", "")
        biotype = tx.get("biotype") or mane_dict.get(tx_id, "")
        if tx.get("isManeSelect") and biotype == "mRNA" and tx_id in mane_dict:
            return tx
    # Step 2: fallback to canonical mRNA
    for tx in transcripts:
        tx_id = tx.get("transcript", "")
        biotype = tx.get("biotype") or mane_dict.get(tx_id, "")
        if biotype == "mRNA" and tx.get("isCanonical"):
            return tx
    # Step 3: return first transcript (best-effort fallback)
    return transcripts[0]

def format_exon_intron(tx):
    """
    Return exon/intron label for the transcript.
    """
    if not tx:
        return ""
    if tx.get("introns"):
        return "intron " + str(tx["introns"])
    if tx.get("exons"):
        return "exon " + str(tx["exons"])
    return ""

def clinvar_filter_for_variant(clinvar_list, ref, alt):
    """
    From ClinVar entries attached to a variant, return only those that match
    the exact ref/alt alleles for this variant.

    Returns three comma-separated strings:
      - IDs
      - significance
      - reviewStatus
    """
    ids, signifs, reviews = [], [], []
    for c in clinvar_list or []:
        # Only include ClinVar entries that match the variant's ref and alt alleles
        if c.get("refAllele") == ref and c.get("altAllele") == alt:
            ids.append(c.get("id", ""))
            signifs.append(safe_join_list(c.get("significance", []), ";"))
            reviews.append(c.get("reviewStatus", ""))
    return safe_join_list(ids, ","), safe_join_list(signifs, ","), safe_join_list(reviews, ",")

def extract_cosmic_ids(var):
    """
    Normalize COSMIC identifiers for a variant.
    """
    cosmic = var.get("cosmic", [])
    out = []
    for c in cosmic or []:
        if isinstance(c, dict):
            for key in ("id", "cosmicId", "cosmic_id"):
                if key in c:
                    out.append(str(c.get(key)))
                    break
        else:
            out.append(str(c))
    return safe_join_list(out, ",")

def global_minor_af_if_matches(var, alt):
    """
    If the variant's alt allele equals the reported globalMinorAllele,
    return the globalMinorAlleleFrequency; otherwise return blank.
    """
    ga = var.get("globalAllele", {})
    if not ga:
        return ""
    if str(ga.get("globalMinorAllele")) == str(alt):
        return ga.get("globalMinorAlleleFrequency", "")
    return ""

def parse_json_to_tsv(json_file, mane_dict):
    """
    Parse a single Nirvana JSON and write a parsed_<basename>.tsv file.
    Important behaviors:
      - Only positions with "PASS" in their filters are exported.
      - Each 'variant' object under a position becomes its own TSV row.
      - Exactly one transcript per variant is selected via select_transcript().
    """
    base_name = os.path.basename(json_file).split(".")[0]
    out_tsv = f"parsed_{base_name}.tsv"
    print(f"\n🔹 Processing {json_file} → {out_tsv}")

    with open(json_file, "r") as jf:
        data = json.load(jf)

    # Columns written to the TSV (order matters)
    header = [
        "chromosome", "position", "refAllele", "altAllele", "filters",
        "mappingQuality", "cytogeneticBand", "variantType", "hgvsg", "dbSNP_ID",
        "clinvar_IDs", "clinvar_significance", "clinvar_reviewStatus",
        "globalMinorAlleleFrequency",
        "gnomad_allAF", "oneKg_allAF", "topmed_allAF",
        "COSMIC_IDs",
        "transcript", "biotype", "exon_intron", "hgnc", "consequence", "hgvsc", "hgvsp",
        "genotype", "variantFrequencies", "totalDepth", "alleleDepths", "somaticQuality"
    ]



    with open(out_tsv, "w", newline='') as outfh:
        writer = csv.writer(outfh, delimiter="\t")
        # Header row
        writer.writerow(header)

        # Iterate positions. Each position represents a genomic site from the VCF.
        for pos in data.get("positions", []):
            # Only include variants at positions that passed filters.
            if "PASS" not in pos.get("filters", []):
                continue

            # Position-level fields
            chrom = pos.get("chromosome", "")
            position = pos.get("position", "")
            ref = pos.get("refAllele", "")
            filters = pos.get("filters", [])
            mappingQuality = pos.get("mappingQuality", "")
            cytoband = pos.get("cytogeneticBand", "")
            samples = pos.get("samples", [])


            for var in pos.get("variants", []):
                alt = var.get("altAllele", "")
                variant_type = var.get("variantType", "")
                hgvsg = var.get("hgvsg", "")
                dbsnp_id = (var.get("dbsnp", []) or [""])[0]

                # ClinVar: only keep entries that match the exact ref/alt pair
                clin_id_str, clin_sig_str, clin_rev_str = clinvar_filter_for_variant(var.get("clinvar", []), ref, alt)

                # Global/minor allele frequency only if alt matches reported minor allele
                gmaf = global_minor_af_if_matches(var, alt)

                # Population frequencies
                gnomad_allaf = var.get("gnomad", {}).get("allAf", "")
                onekg_allaf = var.get("oneKg", {}).get("allAf", "")
                topmed_allaf = var.get("topmed", {}).get("allAf", "")

                # COSMIC IDs normalized
                cosmic_ids = extract_cosmic_ids(var)

                # Choose exactly one transcript for reporting for this variant
                tx = select_transcript(var.get("transcripts", []), mane_dict)
                if tx:
                    tx_name = tx.get("transcript", "")
                    # If transcript has no explicit biotype in JSON, consult MANE GTF lookup
                    biotype = tx.get("biotype") or mane_dict.get(tx_name, "")
                    exon_intron = format_exon_intron(tx)
                    hgnc = tx.get("hgnc", "")
                    consequence = safe_join_list(tx.get("consequence", []))
                    hgvsc = tx.get("hgvsc", "")
                    hgvsp = tx.get("hgvsp", "")
                else:
                    tx_name = biotype = exon_intron = hgnc = consequence = hgvsc = hgvsp = ""

                # Collect per-sample metrics
                genotypes, vfreqs, depths, ad_list, sq_list = [], [], [], [], []
                for s in samples or []:
                    genotypes.append(s.get("genotype", ""))
                    vfs = s.get("variantFrequencies", [])
                    vfreqs.append(",".join(map(str, vfs)) if vfs else "")
                    depths.append(str(s.get("totalDepth", "")))
                    ad = s.get("alleleDepths", [])
                    ad_list.append(",".join(map(str, ad)) if ad else "")
                    sq_list.append(str(s.get("somaticQuality", "")))

                # Write exactly one TSV row for this variant
                writer.writerow([
                    chrom, position, ref, alt, safe_join_list(filters, ","),
                    mappingQuality, cytoband, variant_type, hgvsg, dbsnp_id,
                    clin_id_str, clin_sig_str, clin_rev_str, gmaf,
                    gnomad_allaf, onekg_allaf, topmed_allaf, cosmic_ids,
                    tx_name, biotype, exon_intron, hgnc, consequence, hgvsc, hgvsp,
                    safe_join_list(genotypes), ";".join(vfreqs),
                    safe_join_list(depths), ";".join(ad_list), safe_join_list(sq_list)
                ])

    print(f"✅ Wrote: {out_tsv}")

def main():
    # Load MANE lookup once
    mane_dict = load_mane_transcripts(mane_gtf_file)

    # Collect JSON files in the configured input folder.
    json_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]
    if not json_files:
        print("No JSON files found in folder:", input_folder)
        return

    # Process each JSON file found, producing one parsed_<basename>.tsv output per JSON.
    for jf in json_files:
        parse_json_to_tsv(os.path.join(input_folder, jf), mane_dict)

if __name__ == "__main__":
    main()


Loaded 19404 MANE transcripts from MANE.GRCh38.v1.4.refseq_genomic.gtf

🔹 Processing /Users/kedarjoshi/PyCharmMiscProject/No_Backbone/GX22-1031_017.hard-filtered.vcf.annotated.json → parsed_GX22-1031_017.tsv
✅ Wrote: parsed_GX22-1031_017.tsv

🔹 Processing /Users/kedarjoshi/PyCharmMiscProject/No_Backbone/GX24-2832_024.hard-filtered.vcf.annotated.json → parsed_GX24-2832_024.tsv
✅ Wrote: parsed_GX24-2832_024.tsv

🔹 Processing /Users/kedarjoshi/PyCharmMiscProject/No_Backbone/GX24-1835_019.hard-filtered.vcf.annotated.json → parsed_GX24-1835_019.tsv
✅ Wrote: parsed_GX24-1835_019.tsv

🔹 Processing /Users/kedarjoshi/PyCharmMiscProject/No_Backbone/GX23-389_021.hard-filtered.vcf.annotated.json → parsed_GX23-389_021.tsv
✅ Wrote: parsed_GX23-389_021.tsv

🔹 Processing /Users/kedarjoshi/PyCharmMiscProject/No_Backbone/HD798_020.hard-filtered.vcf.annotated.json → parsed_HD798_020.tsv
✅ Wrote: parsed_HD798_020.tsv

🔹 Processing /Users/kedarjoshi/PyCharmMiscProject/No_Backbone/GX22-1032_018.hard-filtere